In [6]:
from mysql.connector import MySQLConnection
from contextlib import closing
import os
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [10]:
dbconfig = {
    'user': os.getenv('MYSQL_USER'),
    'host': 'localhost',
    'port': '3306',
    'database': 'LittleLemonDB',
    'password': os.getenv('MYSQL_PASSWORD')
}

In [11]:
def execute_dmc(query):
    with closing(
        MySQLConnection(**dbconfig)
    ) as connection:
        with closing(
            connection.cursor()
        ) as cursor:
            cursor.execute(
                query
            )
            connection.commit()

In [12]:
def execute_dql(query):
    with closing(
        MySQLConnection(**dbconfig)
    ) as connection:
        with closing(
            connection.cursor()
        ) as cursor:
            cursor.execute(query)
            return pd.DataFrame(
                data=cursor.fetchall(),
                columns=cursor.column_names
            )

In [13]:
print(
    os.getenv('MYSQL_PASSWORD')
)

None
